# Part II
# Mahla Entezari 

In [1]:
# pip install nltk

In [2]:
import os
import re
import spacy
import nltk
import pandas as pd
from nltk.corpus import names
from collections import Counter
from nltk.corpus import stopwords
from sklearn.cluster import DBSCAN
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.chunk import tree2conlltags
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder
from nltk import word_tokenize, pos_tag, ne_chunk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

In [4]:
# nltk.download('punkt')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('averaged_perceptron_tagger')

# nltk.download('stopwords')

In [5]:
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 15000000

In [6]:
# stemmer = PorterStemmer()

In [7]:
def clean_text(text):
    # text = re.sub(r"\b(\w+)'s\b", r"\1", text)
    # text = re.sub(r"[^\w\s]", "", text)
    # text = re.sub(r'\W', ' ', text)
    # text = text.lower()
    # tokens = nltk.word_tokenize(text)
    # tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    # output = ' '.join(tokens)
    # print(output)
    # return output

    return text

In [8]:
texts = {}
titles = ["01 Harry Potter and the Sorcerers Stone.txt",
              "02 Harry Potter and the Chamber of Secrets.txt",
            #   "03 Harry Potter and the Prisoner of Azkaban.txt",
            #   "04 Harry Potter and the Goblet of Fire.txt",
            #   "05 Harry Potter and the Order of the Phoenix.txt",
            #   "06 Harry Potter and the Half-Blood Prince.txt",
            #   "07 Harry Potter and the Deathly Hallows.txt"
]
data_folder_path = 'HarryPotter_data/'
data_files = os.listdir(data_folder_path)
all_texts = ''
for file_ in data_files:
    book = file_.strip()
    file_path = os.path.join(data_folder_path, file_)
    with open(file_path, 'r', encoding='utf-8') as file:
        temp_text = file.read()
        texts[book] = clean_text(temp_text)
        all_texts += '\n' + temp_text
texts.keys()
# texts
# all_texts = [char for sublist in texts.values() for char in sublist]
# texts_string = ''.join(all_texts)
all_texts = clean_text(all_texts)
all_texts

"\nM r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were\nAnd don’t get in a flap!\n\nYou’re in safe hands (though I have none)\n\nFor I’m a Thinking Cap!”\n\n\n\nThe whole hall burst into applause as the hat finished its song. It bowed to each of the four tables and then became quite still again.\n\n“So we’ve just got to try on the hat!” Ron whispered to Harry. “I’ll kill Fred, he was going on about wrestling a troll.”\n\nHarry. smiled weakly. Yes, trying on the hat was a lot better than having to do a spell, but he did wish they could have tried it on without everyone watching. The hat seemed to be asking rather a lot; Harry didn’t feel brave or quick-witted or any of it at the moment. If only the hat had mentioned a house for people who felt a bit queasy, that would have been the one for him.\n\nProfessor McGonagall now stepped forward holding a long roll of parchment.\n\n“When I call your name, you wil

In [9]:
def get_characters_nlp(text):
    doc = nlp(text)
    characters = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return characters

# characters_nlp = {}
# for book_title, text in texts.items():
#     characters_nlp[book_title] = get_characters_nlp(text)

# all_characters = [char for sublist in characters_nlp.values() for char in sublist]
# characters_string = ' '.join(all_characters)
all_characters = get_characters_nlp(all_texts)


In [10]:
# characters_nlp[titles[0]][:100]

In [11]:
# all_characters = list(set(all_characters))
all_characters

['Dursley',
 'Ron',
 'Harry',
 'Fred',
 'Harry',
 'Harry',
 'McGonagall',
 'Hannah',
 'Hannah',
 'Harry',
 'Susan',
 'Susan',
 'Hannah',
 'Terry',
 'Ravenclaws',
 'Terry',
 'Harry',
 'Ron',
 'Harry',
 'Dudley',
 'Harry',
 'Harry',
 'Ron',
 'Harry',
 'McGonagall',
 'Neville Longbottom',
 'Malfoy',
 'Malfoy',
 'Crabbe and Goyle',
 'Harry',
 'Harry',
 'Harry',
 'Hmm',
 'Harry',
 'Harry',
 'Gryffindor',
 'Harry',
 'Harry',
 'Harry',
 'Albus Dumbledore',
 'Harry',
 'Dumbledore',
 'Harry',
 'Quirrell',
 'Thomas, Dean',
 'Ron',
 'Harry',
 'Gryffindor',
 'Lisa',
 'Ron',
 'Harry',
 'Harry',
 'Ron',
 'Ron',
 'Percy Weasley',
 'Harry',
 'McGonagall',
 'Harry',
 'Albus Dumbledore',
 'Harry',
 'Percy',
 'Percy',
 'Harry',
 'Harry',
 'Yorkshire',
 'Harry',
 'Dudley',
 'Harry',
 'Harry',
 'Harry',
 'Nicholas de Mimsy-Porpington',
 'Ron',
 'Nicholas de Mimsy',
 'Seamus Finnigan',
 'Nicholas',
 'Harry',
 'Malfoy',
 'Harry',
 'Seamus',
 'Jell',
 'Harry',
 'Seamus',
 'Neville',
 'Ron',
 'Neville',
 'Gran

In [12]:
character_counts = Counter(all_characters)

print(character_counts.most_common(100))

[('Harry', 531), ('Ron', 228), ('McGonagall', 52), ('Malfoy', 44), ('Gryffindor', 32), ('Dumbledore', 30), ('Percy', 24), ('Filch', 19), ('Quirrell', 13), ('Hooch', 13), ('Hermione', 13), ('Norris', 11), ('Flitwick', 11), ('Hermione Granger', 10), ('Potter', 10), ('Dean', 10), ('Gringotts', 8), ('Nicolas Flamel', 8), ('Fred', 7), ('Percy Weasley', 7), ('George', 7), ('Flamel', 7), ('Seamus Finnigan', 6), ('Seamus', 6), ('Neville', 6), ('Weasley', 6), ('Charlie', 6), ('Lee Jordan', 6), ('Quaffle', 6), ('Goyle', 5), ('Fred and George', 5), ('Harry, Ron', 5), ('Oliver Wood', 5), ('Hogwarts', 5), ('Muggle', 5), ('Tom', 5), ('Dudley', 4), ('Quidditch', 4), ('Hannah', 3), ('Neville Longbottom', 3), ('Albus Dumbledore', 3), ('Harry Potter', 3), ('Harry quietly', 3), ('Draco Malfoy', 3), ('Dean Thomas', 3), ('Seeker', 3), ('George Weasley', 3), ('Harry and Ron', 3), ('Wood', 3), ('Marcus Flint', 3), ('Pince', 3), ('Weasleys', 3), ('Ginny', 3), ('Lockhart', 3), ('Dippet', 3), ('Susan', 2), ('Te

In [13]:
# def get_characters_nltk(text):
#     tokenized = word_tokenize(text)
#     tagged = pos_tag(tokenized)
#     chunked = ne_chunk(tagged)
#     iob_tagged = tree2conlltags(chunked)
#     print('tokenized:',tokenized)
#     print('tagged:',tagged)
#     print('chunked:',chunked)
#     print('iob_tagged:',iob_tagged)
#     characters = [word for word, pos, ner in iob_tagged if ner == 'PERSON']
#     return characters

In [14]:
# characters_nltk = {}
# for book_title, text in texts.items():
#     print(book_title)
#     characters_nltk[book_title] = get_characters_nltk(text)
#     print(characters_nltk[book_title])

In [15]:
# characters_nltk = characters_nltk[titles[1]][:100]
# characters_nltk

In [16]:
# def get_houses(characters, map):
#     pairs = []
#     for name in characters:
#         house = map.get(name, "Other")
#         pairs.append((name, house))
#     return pairs

# chars = characters_nlp['01 Harry Potter and the Sorcerers Stone.txt'][:100]
# char_houses = get_houses(chars, samples)

# print(char_houses)

In [17]:
def get_character_mentions(text, character_names):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    # print(sentences, '.....')

    character_mentions = {name: [] for name in character_names}

    for sentence in sentences:
        for character in character_names:
            if character in sentence:
                character_mentions[character].append(sentence)
                # print(sentence)
    return character_mentions


In [18]:
book_title = titles[0]
char_mentions = get_character_mentions(all_texts, all_characters)

print(f"Mentions of Harry: {char_mentions['Harry'][:3]}")

Mentions of Harry: ['Ron whispered to Harry.', 'Ron whispered to Harry.', 'Ron whispered to Harry.']


In [19]:
def get_descriptions(char_mentions, characters):
    adjectives = []
    keywords = []
    for sent in sentences:
        doc = nlp(sent)
        for token in doc:
            if token.pos_ == "ADJ":
                adjectives.append(token.text)
            if token.pos_ in ["NOUN", "PROPN"] and token.text.lower() not in characters:
                keywords.append(token.text)
    return adjectives, keywords

# char_descs = get_descriptions(char_mentions)


In [20]:
character_descriptions = {}

for character, sentences in char_mentions.items():
    adjectives, keywords = get_descriptions(sentences, all_characters)
    character_descriptions[character] = {
        "adjectives": Counter(adjectives).most_common(10),
        "keywords": Counter(keywords).most_common(10)
    }

In [21]:
character_descriptions

{'Dursley': {'adjectives': [('proud', 1),
   ('normal', 1),
   ('fantastic', 1),
   ('feeble', 1),
   ('little', 1),
   ('flimsy', 1)],
  'keywords': [('Dursleys', 5),
   ('M', 1),
   ('r.', 1),
   ('Mrs.', 1),
   ('Dursley', 1),
   ('number', 1),
   ('Privet', 1),
   ('Drive', 1),
   ('Harry', 1),
   ('books', 1)]},
 'Ron': {'adjectives': [('first', 1368),
   ('new', 1368),
   ('own', 1368),
   ('old', 1140),
   ('next', 1140),
   ('much', 912),
   ('large', 912),
   ('cold', 912),
   ('more', 912),
   ('right', 912)],
  'keywords': [('Ron', 56088),
   ('Harry', 24396),
   ('Hermione', 10260),
   ('Hagrid', 5016),
   ('.', 2736),
   ('Snape', 2736),
   ('Malfoy', 2736),
   ('time', 2508),
   ('wand', 2280),
   ('Dean', 2052)]},
 'Harry': {'adjectives': [('other', 7965),
   ('first', 6903),
   ('open', 5841),
   ('little', 5310),
   ('good', 4779),
   ('large', 4248),
   ('new', 3717),
   ('horrible', 3717),
   ('cold', 3717),
   ('few', 3717)],
  'keywords': [('Harry', 306918),
   ('R

In [22]:
for character, description in character_descriptions.items():
    print(f"Character: {character}")
    print(f"Adjectives: {description['adjectives']}")
    print(f"Keywords: {description['keywords']}\n")

Character: Dursley
Adjectives: [('proud', 1), ('normal', 1), ('fantastic', 1), ('feeble', 1), ('little', 1), ('flimsy', 1)]
Keywords: [('Dursleys', 5), ('M', 1), ('r.', 1), ('Mrs.', 1), ('Dursley', 1), ('number', 1), ('Privet', 1), ('Drive', 1), ('Harry', 1), ('books', 1)]

Character: Ron
Adjectives: [('first', 1368), ('new', 1368), ('own', 1368), ('old', 1140), ('next', 1140), ('much', 912), ('large', 912), ('cold', 912), ('more', 912), ('right', 912)]
Keywords: [('Ron', 56088), ('Harry', 24396), ('Hermione', 10260), ('Hagrid', 5016), ('.', 2736), ('Snape', 2736), ('Malfoy', 2736), ('time', 2508), ('wand', 2280), ('Dean', 2052)]

Character: Harry
Adjectives: [('other', 7965), ('first', 6903), ('open', 5841), ('little', 5310), ('good', 4779), ('large', 4248), ('new', 3717), ('horrible', 3717), ('cold', 3717), ('few', 3717)]
Keywords: [('Harry', 306918), ('Ron', 55224), ('Hermione', 16992), ('Hagrid', 15930), ('Malfoy', 15399), ('Snape', 14868), ('Potter', 10089), ('eyes', 10089), ('Pro

In [23]:
# char_mentions = get_character_mentions(all_texts, all_characters)
# print('char_mentions',char_mentions)
# char_descs = get_descriptions(char_mentions)
# print('char_descs',char_descs)
# cleaned_descs = [clean_text(desc) for desc in char_descs]
# print('cleaned_descs',cleaned_descs)



In [24]:
# print("Descriptions for a sample:", cleaned_descs[0][:10])

In [25]:
# all_texts

In [26]:
# all_descs = [''.join(desc) for desc in char_descs]
# print(all_descs)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(character_descriptions)


In [27]:
# features = []
# labels = []

# for character, description in character_descriptions.items():
#     if character in samples:
#         features.append(clean_text(description))
#         labels.append(samples[character])

# encoder = LabelEncoder()
# encoded_labels = encoder.fit_transform(labels)

In [28]:
from sklearn.cluster import KMeans

num_clusters = 5

kmeans = KMeans(n_clusters=num_clusters, random_state=42)

In [29]:
data = []
for character, description in character_descriptions.items():
    data.append({
        "character": character,
        "adjectives": description['adjectives'],
        "keywords": description['keywords']
    })

df = pd.DataFrame(data)

# df['description'] = df['adjectives'] + " " + df['keywords']


num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df['house'] = kmeans.fit_predict(X)


In [31]:
most_common_names = [name for name, count in character_counts.most_common(100)]

dff = df[df['character'].isin(most_common_names)]

dff

,character,adjectives,keywords,house
0,Dursley,"[(proud, 1), (normal, 1), (fantastic, 1), (fee...","[(Dursleys, 5), (M, 1), (r., 1), (Mrs., 1), (D...",0
1,Ron,"[(first, 1368), (new, 1368), (own, 1368), (old...","[(Ron, 56088), (Harry, 24396), (Hermione, 1026...",1
2,Harry,"[(other, 7965), (first, 6903), (open, 5841), (...","[(Harry, 306918), (Ron, 55224), (Hermione, 169...",1
3,Fred,"[(last, 14), (nice, 14), (blue, 14), (first, 7...","[(Fred, 126), (George, 70), (Harry, 56), (Weas...",3
4,McGonagall,"[(more, 104), (busy, 104), (redder, 104), (com...","[(Professor, 2808), (McGonagall, 2756), (Harry...",0
...,...,...,...,...
131,Lockhart,"[(last, 3)]","[(Lockhart, 9), (Snape, 3), (Malfoy, 3), (diar...",0
132,Tom Riddle,[],"[(Tom, 4), (Riddle, 4), (name, 2), (Hagrid, 2)...",0
133,Dippet,"[(disappointed, 3)]","[(Dippet, 18), (Professor, 6), (Hogwarts, 6), ...",0
134,Dipper,"[(dear, 2)]","[(Dipper, 4), (boy, 2), (school, 2), (summer, ...",0


In this part, to classify the characters according to their house,\
First, I imported the required libraries.

Then, with the help of pre-trained NER models, I tried to extract the characters from the texts and then find their characteristics and description.

So I took help from NLTK and spacy.

I defined the clean_text function, which takes the text as input and cleans it, that is, for example, it removes conjunctions and superscripts or dot and comma characters, etc.,\
so that the remaining text contains more important and key words.

After that, I define a function that finds the characters of the texts and then the mention function that shows the mod that each character is mentioned.

After that, according to the mentions of each character, a description containing a few key words of the characters is made.

Finally, I have updated my model, but unfortunately I did not get to test it and check the correctness of the clusters